In [30]:
import torch

from datetime import datetime
from pprint import pprint

from config import *
from data.extract import DetectorDataProvider, LookUpTable
from data.util import count_points_between

%reload_ext autoreload
%autoreload 2

In [32]:
READ_START_DATE = datetime.strptime(CONFIG['READ_START_DATE'], DATE_FORMAT)
READ_END_DATE = datetime.strptime(CONFIG['READ_END_DATE'], DATE_FORMAT)

### Read data from the data set

In [34]:
lut = LookUpTable('Data')

/home/kolusask/TUM/IDP/IDP/src/data/extract.py:88: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return np.unique(np.concatenate([


Check list of all sections

In [35]:
sections = lut.get_sections()
pprint(list(sections)[:10])
print(len(sections))

[('1080', 'Schutterstrasse'),
 ('8602', '8604'),
 ('3090', '37'),
 ('4010', '4240'),
 ('5060', '5090'),
 ('7040', 'Erletsstrasse'),
 ('1080', '5080'),
 ('6022', 'Network Boundary'),
 ('4150', 'Schlüterstrasse'),
 ('8008', 'Network Boundary South')]
236


Extract list of detectors for each section

In [36]:
int_det = lut.get_detectors_per_section()
int_det

,Start,End,Detectors
0,1080,Schutterstrasse,[]
1,Schutterstrasse,1080,"[3(DB1), 4(DB2)]"
2,8602,8604,[]
3,8604,8602,[2(DA2)]
4,3090,37,[]
...,...,...,...
467,4240,4210,"[5(DB1), 6(DB2), 7(DB3)]"
468,5012,5020,"[7(DC1), 8(DC2)]"
469,5020,5012,"[1(DA1), 2(DA2), 3(DB1)]"
470,24,4040,"[1(DA1), 2(DA2)]"


Extract counts for all sections

In [37]:
ddp = DetectorDataProvider('Data')

int_det['Counts'] = int_det.apply(
    lambda sec: ddp.get_counts_entering_section(
        sec['End'], sec['Detectors'], READ_START_DATE, READ_END_DATE
    ),
    axis=1
)
int_det

1080 96 DetCount_20210101.csv


,Start,End,Detectors,Counts
0,1080,Schutterstrasse,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Schutterstrasse,1080,"[3(DB1), 4(DB2)]","[0.0, 5.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
2,8602,8604,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,8604,8602,[2(DA2)],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,3090,37,[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
467,4240,4210,"[5(DB1), 6(DB2), 7(DB3)]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
468,5012,5020,"[7(DC1), 8(DC2)]","[1.0, 6.0, 8.0, 3.0, 5.0, 0.0, 4.0, 3.0, 8.0, ..."
469,5020,5012,"[1(DA1), 2(DA2), 3(DB1)]","[5.0, 4.0, 3.0, 4.0, 1.0, 2.0, 1.0, 1.0, 1.0, ..."
470,24,4040,"[1(DA1), 2(DA2)]","[0.0, 4.0, 2.0, 2.0, 2.0, 3.0, 0.0, 2.0, 1.0, ..."


### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [43]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
torch.save(mat_q, out_path('mat_q.pt'))
assert mat_q.shape == (count_points_between(READ_START_DATE, READ_END_DATE), 2 * len(sections))
mat_q.shape                                                         

torch.Size([32064, 472])
32064 472


torch.Size([32064, 472])